<font face="candara">

## "imdb film reviews"

#### MSDS 7337 - Natural Language Processing - Homework 05  
##### Patrick McDevitt  
##### 22-Oct-2018  

***     

For this project we are requested to :
1. Compile a list of static links (permalinks) to individual user movie reviews from one particular website.
    - This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.   
    a. It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.  
    b. Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.  
    c. Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  
    d. Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.  
2. Extract noun phrase (NP) chunks from your reviews using the following procedure:  
    a. In Python, use BeautifulSoup to grab the main review text from each link.  
    b. Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser.  
    c. You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.  
3. Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).  



<font face="candara">

*** 

For this assignment, the following was accomplished :  
- defined a list of some films from the 80's that are each well appreciated by their fans, even if not all particularly well known
- developed a function to extract review titles, ratings, and the text content for each review form the imdb() functions and interface within python libraries
- used nltk() functions to tokenize, identify parts of speech
- continued with nltk() functions to then extract NounPhrases from the text descriptions
- the list of NPs are displayed in the last output section of this file

(This assignment was completed in collaboration with Preeti Swaminathan)

*** 


In [1]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... import some packages
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize, sent_tokenize

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import urllib.request

import imdb
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... text chunker function
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

def review_chunker (review_text) :

    tokens = nltk.word_tokenize(review_text)
    tagged = nltk.pos_tag(tokens)    
    entities = nltk.chunk.ne_chunk(tagged)
#    print("\nentities = : ", entities)
    
    return (entities)

In [3]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... movie list to extract reviews
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

movie_list = ["Stranger Than Paradise", "The Big Lebowski", "Fargo"]

BASE_URL = "https://www.imdb.com"

In [4]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... create table for collecting results
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

tbl_imdb  = pd.DataFrame(columns = [
    'Movie_Name',
    'Review_Title',
    'Review_Desc',
    'Review_Rating',
    'Review_url'])

i_index=[]
i_index = 0

In [5]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... query IMDB for list of reviews
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

for movie_name in movie_list:
    
    ia = imdb.IMDb()
    
    results = ia.search_movie(movie_name)
    mv = results[0]
    
    url = ia.get_imdbURL(mv)
    
    user_review_url = url + 'reviews/'
    
    response = requests.get(user_review_url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    try:
        reviews = soup.select('a.title')
        links = [a.attrs.get('href') for a in soup.select('a.title')]
        
#        print("reviews list length = ", len(reviews))

        for index in range(0, len(reviews)):
            
            review_title = reviews[index].get_text()
            
            review_desc = "none"
            review_rating = "none"
            review_url = BASE_URL + links[index]
            r_response = requests.get(review_url)
            r_soup = BeautifulSoup(r_response.text, 'lxml')
            
            try:

                d1 = r_soup.find('script', type = 'application/ld+json').text
                data = json.loads(d1)
 #               print("data length = ", len(data))
                                
                try:
                    review_desc = data['reviewBody']
                    
                except Exception as e :
 #                   print("Error review extract : " + str(e))
                    review_desc = "NA"
                    
                try:
                    review_rating = data['reviewRating']['ratingValue']
                    
                except Exception as e :
 #                   print("Error review rating : " + str(e))
                    review_rating = "NA"
                            
            except Exception as e :
                continue
 #               print("Error text extraction : " + str(e))
                
            raw_data = {'Movie_Name' : movie_name,
                    'Review_Title' : review_title,
                    'Review_Desc' : review_desc,
                    'Review_Rating' : review_rating,
                    'Review_url' : links[index]}

            df_tbl = pd.DataFrame(raw_data,
                    columns = [
                        'Movie_Name',
                        'Review_Title',
                        'Review_Desc',
                        'Review_Rating',
                        'Review_url'], index = [i_index + 1])

            tbl_imdb = tbl_imdb.append(df_tbl)
            
    except Exception as e:
        continue
#        print("Error acquire links : " + str(e))

In [6]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... table ouput of collected reviews / urls
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

tbl_imdb.reset_index(drop = True, inplace = True)
tbl_imdb[0:9]

Movie_Name                                       Review_Title  \
0  Stranger Than Paradise                 Still good after all these years\n   
1  Stranger Than Paradise                       I'm choking the alligator.\n   
2  Stranger Than Paradise                        "I put a spell on you..."\n   
3  Stranger Than Paradise            A seductive character and mileu study\n   
4  Stranger Than Paradise                       Absorbing Film By Jarmusch\n   
5  Stranger Than Paradise                      Neat little formal exercise\n   
6  Stranger Than Paradise                       A small, sweet masterpiece\n   
7  Stranger Than Paradise   Ironic and Weird Tale of Emptiness and Boredom\n   
8  Stranger Than Paradise                      Beautiful uncomfortableness\n   

                                         Review_Desc Review_Rating  \
0  I just finished watching Stranger Than Paradis...            10   
1  Odd and inspiring.  This film rings true with ...             8   
2  Stranger Than Paradise, which put filmmaker Ji...             9   
3  Reading over the comments so far, it seems tha...            NA   
4  An excellent example of why independent films ...             8   
5  Watched for the second time the other night, a...            NA   
6  Jarmusch was never much of a guy to dip in the...             8   
7  The New World: The teenager Eva Molnar (Eszter...             8   
8  "Stranger than Paradise" (1984): Jim Jarmusch'...            NA   

                       Review_url  
0  /review/rw1230354/?ref_=tt_urv  
1  /review/rw0215543/?ref_=tt_urv  
2  /review/rw1141927/?ref_=tt_urv  
3  /review/rw0215523/?ref_=tt_urv  
4  /review/rw0215525/?ref_=tt_urv  
5  /review/rw0215538/?ref_=tt_urv  
6  /review/rw1013974/?ref_=tt_urv  
7  /review/rw2789286/?ref_=tt_urv  
8  /review/rw1386758/?ref_=tt_urv

In [7]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... NP-chunking and print list
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

# ... grammar rules

NP_grammar = r"""
  NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
      {<NNP>+}                # chunk sequences of proper nouns
      {<DT>?<JJ.*>*<NN.*>+}
      {<PRP$><NNP>+<NN>}
      {<DT><NN><VBG>}
      {<CD><NN><TO><CD><NN>}
      {<JJR><IN><CD><NN>}
      {<DT><JJS><VBG><NNS>}
"""

# ... each title / each review

for this_title, this_review, this_descr in zip(tbl_imdb['Movie_Name'], tbl_imdb['Review_Title'], tbl_imdb['Review_Desc']) :
    
    print ("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    print ("\t\t", this_title, " | Review :",  this_review)
    print ("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    
    entities = review_chunker(this_descr)

    cp = nltk.RegexpParser(NP_grammar)
    result = cp.parse(entities)

    for subtree in result.subtrees():
        if subtree.label() == 'NP': print(subtree)



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
		 Stranger Than Paradise  | Review :  Still good after all these years

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
(NP the/DT first/JJ time/NN)
(NP year/NN)
(NP release/NN)
(NP the/DT film/NN)
(NP fondness/NN)
(NP Several/JJ years/NNS)
(NP the/DT movie/NN)
(NP soundtrack/NN)
(NP aback/NN)
(NP The/DT haunting/NN)
(NP pieces/NNS)
(NP some/DT time/NN)
(NP the/DT album/NN)
(NP the/DT movie/NN)
(NP The/DT only/JJ piece/NN)
(NP popular/JJ music/NN)
(NP the/DT film/NN)
(NP Spell/NNP)
(NP the/DT soundtrack/NN)
(NP mainstream/JJ movies/NNS)
(NP songs/NNS)
(NP such-like/NN)
(NP music/NN)
(NP situ/NN)
(NP the/DT soundtrack/NN)
(NP standalone/NN)
(NP pieces/NNS)
(NP The/DT movie/NN)
(NP a/DT masterpiece/NN)
(NP white/JJ images/NNS)
(NP a/DT starkness/NN)
(NP a/DT clarity/NN)
(NP the/DT alienation/NN)
(NP self/NN)
(NP a/DT land/NN)
(NP the/DT new/JJ hope/NN)
(NP immigrants/NNS)
(NP old/JJ world/NN)
(NP gh

(NP sense/NN)
(NP embarrassed/JJ compassion/NN)
(NP a/DT direct/JJ result/NN)
(NP humanity/NN)
(NP complete/JJ characters/NNS)
(NP a/DT morose/JJ wit/NN)
(NP these/DT proceedings/NNS)
(NP actors/NNS)
(NP a/DT playful/JJ camaraderie/NN)
(NP the/DT motions/NNS)
(NP a/DT natural/JJ friendship/NN)
(NP direction/NN)
(NP a/DT kid/NN)
(NP clothes/NNS)
(NP video/JJ games/NNS)
(NP This/DT honest/NN)
(NP subtext/NN)
(NP Hollywood/NNP)
(NP moments/NNS)
(NP syrupy/JJ tenderness/NN)
(NP mawkish/JJ emotion/NN)
(NP the/DT clichéd/JJ adage/NN)
(NP characters/NNS)
(NP the/DT film/NN)
(NP improvised/JJ quality/NN)
(NP the/DT careful/JJ balance/NN)
(NP static/JJ ugliness/NN)
(NP a/DT subtext/NN)
(NP natural/JJ warmth/NN)
(NP the/DT great/JJ heart/NN)
(NP this/DT film/NN)
(NP characterization/NN)
(NP greatness/NN)
(NP quiet/JJ touch/NN)
(NP films/NNS)
(NP the/DT director/NN)
(NP the/DT awkward/JJ silences/NNS)
(NP every/DT relationship/NN)
(NP the/DT silences/NNS)
(NP character/NN)
(NP development/NN)
(NP

(NP a/DT static/JJ camera/NN)
(NP white/JJ visuals/NNS)
(NP a/DT sense/NN)
(NP the/DT same/JJ time/NN)
(NP The/DT imprecise/NN)
(NP editing/NN)
(NP most/JJS scenes/NNS)
(NP darkness/NN)
(NP use/NN)
(NP Spell/NNP)
(NP bohemian/JJ vibe/NN)
(NP the/DT film/NN)
(NP a/DT few/JJ subtle/JJ cultural/JJ references/NNS)
(NP the/DT fact/NN)
(NP a/DT race/NN)
(NP horse/NN)
(NP the/DT eye/NN)
(NP a/DT whole/JJ lot/NN)
(NP life/NN)
(NP the/DT perfect/JJ film/NN)
(NP nothing/NN)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
		 Stranger Than Paradise  | Review :  Existential quandary

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
(NP immigrating/NN)
(NP cousin/NN)
(NP ten/JJ days/NNS)
(NP relative/NN)
(NP place/NN)
(NP heritage/NN)
(NP stay/NN)
(NP rude/JJ behavior/NN)
(NP bond/NN)
(NP A/DT year/NN)
(NP goofy/NN)
(NP friend/NN)
(NP a/DT car/NN)
(NP spite/NN)
(NP lack/NN)
(NP interest/NN)
(NP the/DT area/NN)
(NP the/DT trio/NN)
(NP vacation/NN)
(NP paradise/

(NP films/NNS)
(NP Raising/NNP Arizona/NNP)
(NP Fargo/NNP)
(NP film/NN)
(NP complete/JJ breakthrough/NN)
(NP success/NN)
(NP winners/NNS)
(NP Big/NNP Lebowski/NNP)
(NP a/DT film/NN)
(NP the/DT many/JJ other/JJ sporadic/JJ jokes/NNS)
(NP laughter/NN)
(NP The/DT film/NN)
(NP mistaken/JJ identity/NN)
(NP eccentric/JJ characters/NNS)
(NP a/DT soiled/JJ rug/NN)
(NP This/DT film/NN)
(NP the/DT bowler/NN)
(NP the/DT allies/NNS)
(NP the/DT pins/NNS)
(NP an/DT actual/JJ bowling/NN)
(NP ball/NN)
(NP This/DT film/NN)
(NP nihilists/NNS)
(NP feminists/NNS)
(NP millionares/NNS)
(NP paedophiles/NNS)
(NP hippies/NNS)
(NP underachieving/JJ students/NNS)
(NP incompetent/JJ criminals/NNS)
(NP pornographers/NNS)
(NP veterans/NNS)
(NP This/DT movie/NN)
(NP anything/NN)
(NP anything/NN)
(NP Some/DT people/NNS)
(NP absurd/JJ looniness/NN)
(NP a/DT bowler/NN)
(NP a/DT tight/JJ purple/NN)
(NP suit/NN)
(NP a/DT bowling/NN)
(NP ball/NN)
(NP finger/NN)
(NP hole/NN)
(NP the/DT camera/NN)
(NP name/NN)
(NP video/NN)

(NP the/DT second/JJ season/NN)
(NP some/DT people/NNS)
(NP some/DT real/JJ piece/NN)
(NP art/NN)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
		 Fargo  | Review :  Best new show on TV

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
(NP Breaking/NNP Bad/NNP)
(NP i/NNS)
(NP something/NN)
(NP share/NN)
(NP the/DT good/JJ news/NN)
(NP This/DT show/NN)
(NP episodes/NNS)
(NP The/DT story/NN)
(NP the/DT photography/NN)
(NP the/DT surreal/JJ landscape/NN)
(NP the/DT comedy/NN)
(NP the/DT drama/NN)
(NP the/DT suspense/NN)
(NP all/DT excellent/NN)
(NP overboard/NN)
(NP this/DT show/NN)
(NP the/DT movie/NN)
(NP something/NN)
(NP The/DT whole/JJ cast/NN)
(NP an/DT amazing/JJ job/NN)
(NP the/DT guy/NN)
(NP the/DT role/NN)
(NP The/DT quiet/JJ menacing/NN)
(NP look/NN)
(NP the/DT relaxed/JJ confident/JJ way/NN)
(NP town/NN)
(NP the/DT strings/NNS)
(NP everyone/NN)
(NP dance/NN)
(NP tune/NN)
(NP devil/NN)
(NP attitude/NN)
(NP smile/NN)
(NP Emmy/NNP)
(NP co